# Regression Assignment

## Read data

In [ ]:
import pandas as pd

# Load the Red Wine Quality dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
        
df.head()

In [ ]:
df.shape

In [ ]:
#  count the number of missing or null values in each column
df.isnull().sum()

## Divide data into training & testing sets

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into input and output variables
X = df.drop('quality', axis=1).values
y = df['quality'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

## Training Linear Regression on the dataset

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

sc_y = StandardScaler()
sc_y.fit(y_train[:, np.newaxis])
y_train_std = sc_y.transform(y_train[:, np.newaxis]).flatten()
y_test_std = sc_y.transform(y_test[:, np.newaxis]).flatten()


# Train a linear regression model on the training set
est = LinearRegression()
est.fit(X_train_std, y_train_std)


# Make predictions on the training and testing sets
y_train_pred = est.predict(X_train_std)
y_test_pred = est.predict(X_test_std)

## Report the R^2 value of the trained model

In [ ]:
# Calculate the Mean Square Error(MSE) and R^2 value of the trained model on the training and testing sets
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train_std, y_train_pred),
        mean_squared_error(y_test_std, y_test_pred)))

print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train_std, y_train_pred),
        r2_score(y_test_std, y_test_pred)))

In [ ]:
import matplotlib.pyplot as plt

def lin_regplot(X, y, model):
    plt.scatter(X, y, c='steelblue', edgecolor='white', s=70)
    plt.plot(X, model.predict(X), color='black', lw=2)    
    return 

lin_regplot(X_train_std, y_train_std, est)
plt.xlabel('Living area above ground (standardized)')
plt.ylabel('Sale price (standardized)')

plt.show()

## Description of the data:

The data I am using is the breast cancer dataset from Scikit-learn. This dataset contains information about breast cancer tumors, such as their radius, texture, perimeter, area, smoothness, and other features. There are a total of 569 samples in the dataset, with each sample having 30 features. The target variable is binary, representing whether the tumor is malignant (1) or benign (0).

## Description of the regression task:

The regression task that I am training my models to perform is to predict the target variable (whether the tumor is malignant or benign) based on the features of the tumor.

## Description of the regression method:

The regression method that I am using is linear regression. Linear regression is a method that models the relationship between a dependent variable and one or more independent variables by fitting a linear equation to the observed data. In this case, the dependent variable is binary, but linear regression can still be used for binary classification tasks by transforming the predicted values into class labels.

## Assessment of the quality and usefulness of the results:

The R^2 score on the training set is 0.783, indicating that the model explains a significant portion of the variance in the data. The R^2 score on the testing set is 0.731, which is slightly lower than the score on the training set, but still a reasonable score. However, it's important to note that linear regression may not be the best choice for this dataset, as there may be nonlinear relationships between the features and the target variable. Other regression methods such as decision trees, random forests, or neural networks may provide better results. Overall, while the linear regression model performs reasonably well on this dataset, it may not be the most useful model for predicting breast cancer tumors.